In [2]:
import pandas as pd
import yaml
import sqlite3
import numpy as np

In [3]:
orders_df = pd.read_csv("orders.csv")
customer_stats_df = pd.read_csv("customer_statistics.csv")

with open("customer_demographics.yaml", "r") as file:
    data = yaml.safe_load(file)

cd_df = pd.DataFrame.from_dict(data, orient="index")

In [ ]:
#orders_df outliers
orders_df_outliers = orders_df[np.abs(orders_df['total'] - orders_df['total'].mean()) > 1.5 * orders_df['total'].std()]

#customer_stats_df
customer_stats_df_outliers = customer_stats_df[np.abs(customer_stats_df['total_orders'] - customer_stats_df['total_orders'].mean()) > 1.5 * customer_stats_df['total_orders'].std()]

#total items_outliers
total_items_outliers = customer_stats_df[np.abs(customer_stats_df['total_items'] - customer_stats_df['total_items'].mean()) > 1.5 * customer_stats_df['total_items'].std()]

#total spent_outliers
total_spent_outliers = customer_stats_df[np.abs(customer_stats_df['total_spent'] - customer_stats_df['total_spent'].mean()) > 1.5 * customer_stats_df['total_spent'].std()]


print(orders_df_outliers)
print(customer_stats_df_outliers)
print(total_items_outliers)
print(total_spent_outliers)

In [5]:
orders_df.dropna(inplace=True)
orders_df.drop_duplicates(inplace=True)

cd_df.dropna(inplace=True, subset="customer_id")
cd_df.drop_duplicates(inplace=True, subset="customer_id")

In [ ]:
merged = pd.merge(customer_stats_df, cd_df, left_index=True, right_index=True)

merged.drop_duplicates(inplace=True, subset="customer_id_x")
merged

In [ ]:
all_merged_data = pd.merge(merged, orders_df, left_on='customer_id_x', right_on='customer_id')
all_merged_data

## DATABASE STUFF

In [12]:
conn = sqlite3.connect("restaurant.db")

cursor = conn.cursor()

In [13]:
customers_stats_table = """
create table customer_stats(
    customer_index integer primary key autoincrement,
    customer_id integer,
    total_orders integer,
    total_items integer,
    total_spent real
)
"""

cursor.execute(customers_stats_table)

In [14]:
customer_stats_cols = merged.iloc[:, :4]
customer_stats_data_as_list = [tuple(entry) for entry in customer_stats_cols.values]

cursor.executemany("insert into customer_stats(customer_id, total_orders, total_items, total_spent) values (?, ?, ?, ?)", customer_stats_data_as_list)

In [15]:
orders_table = """
create table orders(
    order_id text primary key,
    customer_id integer,
    item integer,
    aperitifs text,
    appetizers text,
    entrees text,
    desserts text,
    total real,
    foreign key(customer_id) references customer_stats(customer_id)
)
"""

cursor.execute(orders_table)

In [16]:
orders_data_as_list = [tuple(entry) for entry in orders_df.values]

cursor.executemany("insert into orders values (?, ?, ?, ?, ?, ?, ?, ?)", orders_data_as_list)

In [17]:
customer_demographics_table = """
create table customer_demographics(
    customer_index integer primary key,
    address text,
    city text,
    credit_card_expires text,
    credit_card_number integer,
    credit_card_provider text,
    credit_card_security_code integer,
    customer_id text,
    email text,
    name text,
    phone_number text,
    state text,
    zip_code integer,
    foreign key(customer_index) references customer_stats(customer_index)
)
"""

cursor.execute(customer_demographics_table)

In [18]:
demographic_cols = merged.iloc[:, 4:]
demographic_data_as_list = [tuple(entry) for entry in demographic_cols.values]

cursor.executemany("""insert into customer_demographics(address, city, credit_card_expires, credit_card_number, credit_card_provider, 
                   credit_card_security_code, customer_id, email, name, phone_number, state, zip_code) 
                   values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", demographic_data_as_list)

In [19]:
conn.commit()
conn.close()